In [1]:
import networkx as nx
import pickle

with open("../resources/LegalBases/graph.pkl", "rb") as f:
    g = pickle.load(f)

In [4]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_core.documents import Document

texts = []
metadatas = []
ids = []
for id, doc in g.nodes(data=True):
    if doc["data"].page_content:
        ids.append(id)
        metadatas.append({"label": doc["label"]})
        texts.append(doc["data"].page_content)

In [6]:
from src.system.rag import Retriever
articles = {}
articles["texts"] = texts
articles["ids"] = ids
articles["metadatas"] = metadatas
config_path = "../config/retriever_config.yaml"
local_embeddings = OllamaEmbeddings(model="all-minilm")
retriever = Retriever(config_path, articles, local_embeddings)

In [16]:
from src.system.rag import RAG
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3")

rag_config_path = "../config/config.yaml"
rag = RAG(rag_config_path, llm, retriever, g, None)

In [17]:
query = "What does it take for an idea to be eligible for patenting?"
rag.run(query)

According to the European Patent Office (EPO) guidelines and the European Patent Convention (EPC), for an idea to be eligible for patenting, it must meet the following criteria:

1. It must be an "invention" belonging to any field of technology.
2. The invention must be "susceptible of industrial application".
3. The invention must be "new".
4. The invention must involve an "inventive step".

Additionally, the EPO guidelines emphasize that a technical character is an implicit requisite for the presence of an "invention" within the meaning of Art. 52(1). This means that the idea must relate to a technical field, be concerned with a technical problem, and have technical features in terms of which the matter for which protection is sought can be defined in the claim.

The two-hurdle approach is also relevant here: the first hurdle assesses whether the claimed subject-matter as a whole falls under the exclusions from patentability under Art. 52(2), while the second hurdle assesses inventiv

'According to the European Patent Office (EPO) guidelines and the European Patent Convention (EPC), for an idea to be eligible for patenting, it must meet the following criteria:\n\n1. It must be an "invention" belonging to any field of technology.\n2. The invention must be "susceptible of industrial application".\n3. The invention must be "new".\n4. The invention must involve an "inventive step".\n\nAdditionally, the EPO guidelines emphasize that a technical character is an implicit requisite for the presence of an "invention" within the meaning of Art. 52(1). This means that the idea must relate to a technical field, be concerned with a technical problem, and have technical features in terms of which the matter for which protection is sought can be defined in the claim.\n\nThe two-hurdle approach is also relevant here: the first hurdle assesses whether the claimed subject-matter as a whole falls under the exclusions from patentability under Art. 52(2), while the second hurdle assesse

In [21]:
from pathlib import Path
import os
ROOT_DIR = Path(os.getcwd())
print(ROOT_DIR)
ROOT_DIR = Path(__file__).resolve().parent.parent

/home/timothee/Documents/data_challenges/PatentAssist/notebooks


NameError: name '__file__' is not defined